# Training Regression - Reaction

# Import packages

In [1]:
import os
import sys

current_path=os.getcwd()
print(current_path)

parent_path=os.path.dirname(current_path)
print(parent_path)

if parent_path not in sys.path:
    sys.path.append(parent_path)

/home/labhhc2/Documents/workspace/D20/Tam/repo/chemprop_1/examples
/home/labhhc2/Documents/workspace/D20/Tam/repo/chemprop_1


In [2]:
import pandas as pd
from lightning import pytorch as pl
from pathlib import Path

from chemprop import data, featurizers, models, nn

# Change data inputs here

## Load data

In [3]:
import numpy as np
chemprop_dir = Path.cwd().parent
num_workers = 20  # number of workers for dataloader. 0 means using main process for data loading
# smiles_column = 'AAM'
# target_columns = ['lograte']

## Perform data splitting for training, validation, and testing

## Get ReactionDatasets

In [4]:
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torch_geometric.data import Data, Batch

# --- BƯỚC 1: ĐỊNH NGHĨA DATASET CHO PRE-TRAINING ---

class MaskedFeatureDataset(Dataset):
    """
    Dataset này sẽ:
    1. Tải các features từ file NPZ.
    2. Trong mỗi lần lấy dữ liệu (__getitem__), nó sẽ che ngẫu nhiên một phần 
       của node_attrs và edge_attrs.
    3. Trả về cả dữ liệu gốc và dữ liệu đã bị che.
    """
    def __init__(self, node_attrs, edge_attrs, edge_indices, mask_fraction=0.15):
        self.node_attrs = [torch.tensor(attrs, dtype=torch.float32) for attrs in node_attrs]
        self.edge_attrs = [torch.tensor(attrs, dtype=torch.float32) for attrs in edge_attrs]
        self.edge_indices = [torch.tensor(idx, dtype=torch.long) for idx in edge_indices]
        self.mask_fraction = mask_fraction

    def __len__(self):
        return len(self.node_attrs)

    def __getitem__(self, idx):
        # Lấy dữ liệu gốc
        original_nodes = self.node_attrs[idx]
        original_edges = self.edge_attrs[idx]
        edge_index = self.edge_indices[idx]

        # Tạo bản sao để che
        masked_nodes = original_nodes.clone()
        masked_edges = original_edges.clone()

        # Che ngẫu nhiên một phần node features
        num_node_features_to_mask = int(original_nodes.shape[0] * self.mask_fraction)
        node_mask_indices = torch.randperm(original_nodes.shape[0])[:num_node_features_to_mask]
        masked_nodes[node_mask_indices] = 0.0 # Che bằng cách gán giá trị 0

        # Che ngẫu nhiên một phần edge features
        num_edge_features_to_mask = int(original_edges.shape[0] * self.mask_fraction)
        edge_mask_indices = torch.randperm(original_edges.shape[0])[:num_edge_features_to_mask]
        masked_edges[edge_mask_indices] = 0.0 # Che bằng cách gán giá trị 0
        
        return {
            "masked_nodes": masked_nodes,
            "masked_edges": masked_edges,
            "original_nodes": original_nodes,
            "original_edges": original_edges,
            "node_mask_indices": node_mask_indices,
            "edge_mask_indices": edge_mask_indices,
            "edge_index": edge_index
        }

# --- TẢI VÀ KẾT HỢP TẤT CẢ DỮ LIỆU ĐỂ PRE-TRAIN ---
# Pre-training là tự giám sát nên chúng ta có thể dùng cả train/val/test data

# Tải dữ liệu từ các file NPZ của bạn
train_npz = np.load(f'../chemprop/data/RC/full/barriers_rgd1/barriers_rgd1_aam_train_rc_processed_data.npz', allow_pickle=True)
val_npz = np.load(f'../chemprop/data/RC/full/barriers_rgd1/barriers_rgd1_aam_val_rc_processed_data.npz', allow_pickle=True)
test_npz = np.load(f'../chemprop/data/RC/full/barriers_rgd1/barriers_rgd1_aam_test_rc_processed_data.npz', allow_pickle=True)

# Kết hợp dữ liệu
all_node_attrs = np.concatenate((train_npz['node_attrs'], val_npz['node_attrs'], test_npz['node_attrs']))
all_edge_attrs = np.concatenate((train_npz['edge_attrs'], val_npz['edge_attrs'], test_npz['edge_attrs']))
all_edge_indices = np.concatenate((train_npz['edge_indices'], val_npz['edge_indices'], test_npz['edge_indices']))

print(f"Tổng số mẫu để pre-train: {len(all_node_attrs)}")

def get_reverse_edge_index(edge_index):
    """Tính toán chỉ số của các cạnh ngược."""
    rev_edge_index = torch.zeros_like(edge_index[0])
    for i in range(edge_index.shape[1]):
        # Tìm cạnh ngược (j, i) cho mỗi cạnh (i, j)
        edge_to_find = edge_index[:, i].flip(0)
        # So sánh để tìm vị trí
        matches = (edge_index.T == edge_to_find).all(dim=1)
        # Lấy chỉ số đầu tiên tìm thấy
        rev_idx = torch.where(matches)[0]
        if rev_idx.numel() > 0:
            rev_edge_index[i] = rev_idx[0]
        else:
            # Xử lý trường hợp không tìm thấy cạnh ngược (ít khả năng xảy ra nếu đồ thị là vô hướng)
            rev_edge_index[i] = -1 # hoặc một giá trị đặc biệt
    return rev_edge_index


def collate_pretraining_batch(samples):
    """
    Hàm này sẽ gộp một list các sample (dictionaries) thành một batch duy nhất,
    đồng thời đảm bảo các thuộc tính V, E, và rev_edge_index được đặt tên đúng.
    """
    batch_data = []
    for i, sample in enumerate(samples):
        edge_index = sample["edge_index"]
        
        # *** THÊM BƯỚC NÀY ***
        # Tính toán rev_edge_index
        rev_edge_index = get_reverse_edge_index(edge_index)

        data_point = Data(
            V=sample["masked_nodes"],
            E=sample["masked_edges"],
            edge_index=edge_index,
            
            # Thêm rev_edge_index vào đối tượng Data
            rev_edge_index=rev_edge_index,
            
            original_V=sample["original_nodes"],
            node_mask_indices=sample["node_mask_indices"],
            sample_idx=i
        )
        batch_data.append(data_point)

    return Batch.from_data_list(batch_data)

# Khởi tạo lại DataLoader với collate_fn mới
pretrain_dataset = MaskedFeatureDataset(all_node_attrs, all_edge_attrs, all_edge_indices)
pretrain_loader = DataLoader(pretrain_dataset, batch_size=32, shuffle=True, collate_fn=collate_pretraining_batch)

Tổng số mẫu để pre-train: 353468


In [5]:
# import torch
# import numpy as np
# from torch.utils.data import Dataset
# from torch_geometric.loader import DataLoader 
# from torch_geometric.data import Data, Batch

# # --- ĐỊNH NGHĨA DATASET MỚI CHO DGI ---

# class GraphDataset(Dataset):
#     """
#     Dataset đơn giản cho DGI:
#     1. Tải các features từ file NPZ.
#     2. Chỉ trả về dữ liệu đồ thị gốc, không che (masking).
#     """
#     def __init__(self, node_attrs, edge_attrs, edge_indices):
#         self.node_attrs = [torch.tensor(attrs, dtype=torch.float32) for attrs in node_attrs]
#         self.edge_attrs = [torch.tensor(attrs, dtype=torch.float32) for attrs in edge_attrs]
#         self.edge_indices = [torch.tensor(idx, dtype=torch.long) for idx in edge_indices]

#     def __len__(self):
#         return len(self.node_attrs)

#     def __getitem__(self, idx):
#         # Lọc bỏ các đồ thị rỗng ngay tại đây
#         if self.node_attrs[idx].shape[0] == 0:
#             # Trả về một đồ thị rỗng hợp lệ thay vì None hoặc dictionary
#             return Data()

#         nodes = self.node_attrs[idx]
#         edges = self.edge_attrs[idx]
#         edge_index = self.edge_indices[idx]
        
#         # (Tùy chọn nhưng khuyến khích) Tính rev_edge_index ở đây nếu cần
#         rev_edge_index = get_reverse_edge_index(edge_index)

#         # Tạo và trả về một đối tượng Data hoàn chỉnh
#         data_point = Data(
#             V=nodes,
#             E=edges,
#             edge_index=edge_index,
#             rev_edge_index=rev_edge_index
#         )
#         return data_point

# # --- TẢI DỮ LIỆU (Giữ nguyên) ---
# train_npz = np.load(f'../chemprop/data/normal/barriers_cycloadd/barriers_cycloadd_aam_train_processed_data.npz', allow_pickle=True)
# val_npz = np.load(f'../chemprop/data/normal/barriers_cycloadd/barriers_cycloadd_aam_val_processed_data.npz', allow_pickle=True)
# test_npz = np.load(f'../chemprop/data/normal/barriers_cycloadd/barriers_cycloadd_aam_test_processed_data.npz', allow_pickle=True)

# all_node_attrs = np.concatenate((train_npz['node_attrs'], val_npz['node_attrs'], test_npz['node_attrs']))
# all_edge_attrs = np.concatenate((train_npz['edge_attrs'], val_npz['edge_attrs'], test_npz['edge_attrs']))
# all_edge_indices = np.concatenate((train_npz['edge_indices'], val_npz['edge_indices'], test_npz['edge_indices']))

# print(f"Tổng số mẫu để pre-train: {len(all_node_attrs)}")

# # --- ĐỊNH NGHĨA COLLATE_FN MỚI CHO DGI ---

# # Giữ lại hàm này nếu mô hình BondMessagePassing của bạn cần
# def get_reverse_edge_index(edge_index):
#     # ... (code hàm này giữ nguyên) ...
#     rev_edge_index = torch.zeros_like(edge_index[0])
#     for i in range(edge_index.shape[1]):
#         edge_to_find = edge_index[:, i].flip(0)
#         matches = (edge_index.T == edge_to_find).all(dim=1)
#         rev_idx = torch.where(matches)[0]
#         if rev_idx.numel() > 0:
#             rev_edge_index[i] = rev_idx[0]
#         else:
#             rev_edge_index[i] = -1
#     return rev_edge_index

# def collate_dgi_batch(samples):
#     """
#     Hàm collate cho DGI, gộp các sample thành một batch của PyG.
#     Sử dụng các key tiêu chuẩn: x, edge_index, edge_attr.
#     """
#     batch_data = []
#     for sample in samples:
#         # Lọc bỏ các đồ thị rỗng để tránh lỗi
#         if sample["nodes"].shape[0] == 0:
#             continue
            
#         edge_index = sample["edge_index"]
#         rev_edge_index = get_reverse_edge_index(edge_index)

#         data_point = Data(
#             x=sample["nodes"],          # Key tiêu chuẩn của PyG
#             edge_attr=sample["edges"],  # Key tiêu chuẩn của PyG
#             edge_index=edge_index,
#             rev_edge_index=rev_edge_index
#         )
#         batch_data.append(data_point)

#     if not batch_data:
#         return None

#     return Batch.from_data_list(batch_data)

# # --- KHỞI TẠO DATALOADER MỚI ---
# pretrain_dataset = GraphDataset(all_node_attrs, all_edge_attrs, all_edge_indices)
# pretrain_loader = DataLoader(
#     pretrain_dataset, 
#     batch_size=32, 
#     shuffle=True, 
#     collate_fn=collate_dgi_batch,
#     num_workers=0 # Bắt đầu với 0 để gỡ lỗi
# )

In [6]:
# import torch
# import torch.nn as nn 
# import pytorch_lightning as pl
# from torch_geometric.nn import global_mean_pool

# # --- CÁC THÀNH PHẦN CỦA DGI (ĐÃ SỬA) ---

# class Discriminator(nn.Module):
#     """Bộ phân biệt cho DGI."""
#     def __init__(self, hidden_dim):
#         super().__init__()
#         self.weight = torch.nn.Parameter(torch.Tensor(hidden_dim, hidden_dim))
#         torch.nn.init.xavier_uniform_(self.weight)

#     def forward(self, node_summary, graph_summary):
#         graph_summary_proj = torch.matmul(graph_summary, self.weight)
#         return torch.sum(node_summary * graph_summary_proj, dim=1)


# class DeepGraphInfomax(pl.LightningModule):
#     """
#     Mô hình Lightning cho Deep Graph Infomax.
#     """
#     def __init__(self, gnn_encoder):
#         super().__init__()
#         self.gnn = gnn_encoder
#         # Giả định encoder của bạn có thuộc tính .output_dim
#         # Nếu không, hãy thay thế bằng hidden_dim bạn đã định nghĩa
#         hidden_dim = self.gnn.output_dim 
        
#         self.discriminator = Discriminator(hidden_dim)
#         # SỬA Ở ĐÂY: Dùng nn.BCEWithLogitsLoss() từ torch.nn
#         self.loss_fn = torch.nn.BCEWithLogitsLoss()

#     def forward(self, batch):
#         # Giả định mô hình gnn của bạn có thể nhận batch của PyG
#         # Chemprop MPNN thường nhận batch và tự lấy V, E, rev_edge_index
#         node_embeddings = self.gnn(batch) 
        
#         # Lấy biểu diễn đồ thị (global summary)
#         graph_summary = global_mean_pool(node_embeddings, batch.batch)
        
#         return node_embeddings, graph_summary

#     def training_step(self, batch, batch_idx):
#         if batch is None:
#             return None

#         node_embeddings, graph_summary = self.forward(batch)

#         positive_graph_summary = graph_summary[batch.batch]
        
#         shuffled_graph_summary = torch.roll(graph_summary, shifts=1, dims=0)
#         negative_graph_summary = shuffled_graph_summary[batch.batch]

#         positive_score = self.discriminator(node_embeddings, positive_graph_summary)
#         negative_score = self.discriminator(node_embeddings, negative_graph_summary)

#         loss_pos = self.loss_fn(positive_score, torch.ones_like(positive_score))
#         loss_neg = self.loss_fn(negative_score, torch.zeros_like(negative_score))
        
#         loss = loss_pos + loss_neg
        
#         acc = ( (positive_score > 0).float().sum() + (negative_score < 0).float().sum() ) / (2 * len(positive_score))
#         self.log_dict({'train_loss': loss, 'train_acc': acc}, prog_bar=True)
        
#         return loss

#     def configure_optimizers(self):
#         return torch.optim.Adam(self.parameters(), lr=1e-3)

# Change Message-Passing Neural Network (MPNN) inputs here

## Message passing

Message passing blocks must be given the shape of the featurizer's outputs.

Options are `mp = nn.BondMessagePassing()` or `mp = nn.AtomMessagePassing()`

## Construct MPNN

In [7]:
# # SỬA LẠI LỚP PRETRAININGMPNN

# from torch.nn import Linear, MSELoss

# class PretrainingMPNN(pl.LightningModule):
#     def __init__(self, message_passing, node_feature_dim):
#         super().__init__()
#         self.message_passing = message_passing
#         hidden_dim = self.message_passing.output_dim
#         self.node_prediction_head = Linear(hidden_dim, node_feature_dim)
#         self.loss_fn = MSELoss()

# # Sửa trong lớp PretrainingMPNN

#     def forward(self, batch):
#         # Hàm forward của BondMessagePassing sẽ tự động dùng batch.V và batch.E
        
#         # SỬA Ở ĐÂY: Gán kết quả cho một biến duy nhất
#         node_embeddings = self.message_passing(batch)
        
#         predicted_node_features = self.node_prediction_head(node_embeddings)
#         return predicted_node_features
#     def training_step(self, batch, batch_idx):
#         predicted_nodes = self.forward(batch)
        
#         # Tạo boolean mask cho toàn bộ batch
#         total_nodes = batch.V.shape[0] # Dùng batch.V thay vì batch.x
#         node_mask = torch.zeros(total_nodes, dtype=torch.bool, device=self.device)

#         base_idx = 0
#         for i in range(batch.num_graphs):
#             # Lấy các chỉ số mask cho từng đồ thị con trong batch
#             sample_mask_indices = batch.node_mask_indices[batch.batch == i]
            
#             # Cập nhật mask tổng
#             node_mask[base_idx + sample_mask_indices] = True
            
#             # Di chuyển đến điểm bắt đầu của đồ thị tiếp theo
#             num_nodes_in_sample = (batch.batch == i).sum()
#             base_idx += num_nodes_in_sample
        
#         # Chỉ tính loss trên các node đã bị che
#         if node_mask.sum() > 0:
#             # Dùng batch.original_V để lấy feature gốc
#             loss = self.loss_fn(predicted_nodes[node_mask], batch.original_V[node_mask])
#             self.log("train_loss", loss, prog_bar=True)
#             return loss
            
#         return None

#     def configure_optimizers(self):
#         return torch.optim.Adam(self.parameters(), lr=1e-3)

In [8]:
import torch
from torch import Tensor, nn, optim
from torch.nn import MSELoss
from lightning import pytorch as pl

from chemprop.data import BatchMolGraph
from chemprop.nn import MessagePassing
from chemprop.schedulers import build_NoamLike_LRSched


class Pretrainable_MPNN(pl.LightningModule):
    """
    Phiên bản MPNN được chỉnh sửa cho tác vụ pre-train (tái tạo đặc trưng nút).
    Lớp này chỉ giữ lại bộ mã hóa MessagePassing và thêm vào một đầu dự đoán nút.
    """
    def __init__(
        self,
        message_passing: MessagePassing,
        node_feature_dim: int, # THÊM: Cần biết chiều của đặc trưng nút để tái tạo
        warmup_epochs: int = 2,
        init_lr: float = 1e-4,
        max_lr: float = 1e-3,
        final_lr: float = 1e-4,
    ):
        super().__init__()
        self.save_hyperparameters(ignore=["message_passing"])
        hidden_dim = message_passing.output_dim

        # === SỬA: CÁC KHỐI KIẾN TRÚC ===
        self.message_passing = message_passing
        
        # THÊM: Đầu dự đoán để tái tạo lại đặc trưng nút từ các biểu diễn đã học
        self.node_prediction_head = nn.Linear(hidden_dim, node_feature_dim)
        
        # THÊM: Hàm loss để so sánh nút dự đoán và nút gốc
        self.loss_fn = MSELoss()

        # BỎ: GatedSkipBlock, GRUCell, Predictor, Metrics đã được loại bỏ

    def forward(self, batch: BatchMolGraph) -> Tensor:
        """
        SỬA: Hàm forward giờ đây chỉ thực hiện việc mã hóa và dự đoán nút.
        Nó nhận vào batch dữ liệu đã được che (masked).
        """
        # 1. Chạy Message Passing để học đặc trưng nguyên tử từ input đã bị che
        #    message_passing sẽ tự động dùng batch.V, batch.E, ...
        node_embeddings = self.message_passing(batch)
        
        # 2. Dùng đầu dự đoán để tái tạo lại đặc trưng gốc của các nút
        predicted_node_features = self.node_prediction_head(node_embeddings)
        
        return predicted_node_features

    def training_step(self, batch: BatchMolGraph, batch_idx):
        """
        SỬA: training_step được viết lại hoàn toàn cho tác vụ pre-train.
        """
        # 1. Thực hiện forward pass để lấy các nút đã được dự đoán/tái tạo
        predicted_nodes = self(batch)
        
        # 2. Tạo một mask để xác định các nút nào đã bị che trong toàn bộ batch
        total_nodes = batch.V.shape[0]
        node_mask = torch.zeros(total_nodes, dtype=torch.bool, device=self.device)

        base_idx = 0
        for i in range(batch.num_graphs):
            # Lấy chỉ số các nút bị che của từng đồ thị trong batch
            sample_mask_indices = batch.node_mask_indices[batch.batch == i]
            # Cập nhật vào mask chung của toàn batch
            node_mask[base_idx + sample_mask_indices] = True
            # Di chuyển đến điểm bắt đầu của đồ thị tiếp theo
            num_nodes_in_sample = (batch.batch == i).sum()
            base_idx += num_nodes_in_sample
        
        # 3. Chỉ tính loss trên các nút đã bị che
        if node_mask.sum() > 0:
            # Lấy các đặc trưng gốc từ batch (batch.original_V)
            original_nodes = batch.original_V
            loss = self.loss_fn(predicted_nodes[node_mask], original_nodes[node_mask])
            self.log("train_loss", loss, prog_bar=True, batch_size=batch.num_graphs)
            return loss
            
        return None # Bỏ qua batch nếu không có nút nào bị che

    def configure_optimizers(self):
        # Giữ nguyên logic tối ưu hóa
        opt = optim.Adam(self.parameters(), self.hparams.init_lr)
        
        if self.trainer is None or self.trainer.train_dataloader is None:
            return {"optimizer": opt}
            
        steps_per_epoch = self.trainer.num_training_batches
        warmup_steps = self.hparams.warmup_epochs * steps_per_epoch

        if self.trainer.max_epochs == -1:
            cooldown_steps = 100 * warmup_steps
        else:
            cooldown_epochs = self.trainer.max_epochs - self.hparams.warmup_epochs
            cooldown_steps = cooldown_epochs * steps_per_epoch
            
        lr_sched = build_NoamLike_LRSched(
            opt,
            warmup_steps,
            cooldown_steps,
            self.hparams.init_lr,
            self.hparams.max_lr,
            self.hparams.final_lr
        )
        return {"optimizer": opt, "lr_scheduler": {"scheduler": lr_sched, "interval": "step"}}

    # BỎ: validation_step, test_step, predict_step vì mục tiêu chỉ là pre-train
    # BỎ: Các phương thức load_from_checkpoint phức tạp có thể được đơn giản hóa nếu cần

# Training and testing

## Test results

In [9]:
# --- BƯỚC 3: KHỞI TẠO VÀ HUẤN LUYỆN MÔ HÌNH PRE-TRAINING (ĐÃ SỬA LỖI) ---
from chemprop.nn import BondMessagePassing
# Lấy thông số chiều từ dữ liệu
node_feature_dim = pretrain_dataset.node_attrs[0].shape[1]
edge_feature_dim = pretrain_dataset.edge_attrs[0].shape[1]
fdims = (node_feature_dim, edge_feature_dim)

# Khởi tạo encoder D-MPNN
mp_encoder = BondMessagePassing(*fdims)

# Khởi tạo mô hình pre-training
# SỬA Ở ĐÂY: Xóa `edge_feature_dim` khỏi lời gọi hàm
pretrain_model = Pretrainable_MPNN(
    message_passing=mp_encoder, 
    node_feature_dim=node_feature_dim
)

# Khởi tạo Trainer của PyTorch Lightning
trainer = pl.Trainer(
    max_epochs=50, 
    accelerator="auto",
    devices=1,
    enable_progress_bar=True,
    logger=True
)

# Bắt đầu pre-training!
print("Bắt đầu Pre-training...")
trainer.fit(pretrain_model, pretrain_loader)
print("Pre-training hoàn tất!")


# --- BƯỚC 4: LƯU LẠI ENCODER ĐÃ ĐƯỢC HUẤN LUYỆN ---
output_path = "pretrained_dmpnn_nhap_encoder.pt"
torch.save(pretrain_model.message_passing.state_dict(), output_path)
print(f"Encoder đã pre-train được lưu tại: {output_path}")

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/labhhc2/anaconda3/envs/chemprop/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/logger_connector/logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
You are using a CUDA device ('NVIDIA GeForce RTX 4070 Ti SUPER') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.

Bắt đầu Pre-training...
Epoch 0:   2%|▏         | 225/11046 [00:02<02:18, 78.16it/s, v_num=21, train_loss=7.41e+3]

/home/labhhc2/anaconda3/envs/chemprop/lib/python3.12/site-packages/lightning/pytorch/loops/optimization/automatic.py:132: `training_step` returned `None`. If this was on purpose, ignore this warning...


Epoch 49: 100%|██████████| 11046/11046 [36:12<00:00,  5.09it/s, v_num=21, train_loss=5.74e-5] 

`Trainer.fit` stopped: `max_epochs=50` reached.


Epoch 49: 100%|██████████| 11046/11046 [36:12<00:00,  5.08it/s, v_num=21, train_loss=5.74e-5]
Pre-training hoàn tất!
Encoder đã pre-train được lưu tại: pretrained_dmpnn_nhap_encoder.pt


In [10]:
# # --- HUẤN LUYỆN MÔ HÌNH DGI ---
# from chemprop import nn

# # Lấy thông số chiều từ dữ liệu
# node_feature_dim = pretrain_dataset.node_attrs[0].shape[1]
# edge_feature_dim = pretrain_dataset.edge_attrs[0].shape[1]
# fdims = (node_feature_dim, edge_feature_dim)
# hidden_dim = 300 # Ví dụ

# mp_encoder = nn.BondMessagePassing(*fdims, d_h=hidden_dim)

# # Khởi tạo mô hình DGI với encoder GNN
# pretrain_model = DeepGraphInfomax(gnn_encoder=mp_encoder)

# # Khởi tạo Trainer (giữ nguyên)
# trainer = pl.Trainer(
#     max_epochs=50, 
#     accelerator="auto",
#     devices=1
# )

# # Bắt đầu pre-training!
# print("Bắt đầu Pre-training với Deep Graph Infomax...")
# trainer.fit(pretrain_model, pretrain_loader)
# print("Pre-training hoàn tất!")

# # --- LƯU LẠI ENCODER ---
# output_path = "pretrained_dgi_encoder.pt"
# # Lưu lại gnn encoder, vì đó là phần chúng ta cần cho downstream task
# torch.save(pretrain_model.gnn.state_dict(), output_path)
# print(f"Encoder đã pre-train được lưu tại: {output_path}")